# Using `returns` as a pipeline

A close friend recently asked me about putting together network automation tasks into a pipeline.
There are several ways to do this depending on your needs and requirements.
[Ansible](https://www.ansible.com/) is a popular open-source tool for automating tasks, but it is cumbersome to work with for complex tasks, and not everyone enjoys programming in YAML.
[Nornir](https://github.com/nornir-automation/nornir) is a Python automation tool with a very active community and no DSL, puro Python ❤️.
Plenty of shops have even more complex needs and roll their own framework or API to meet highly specific business requirements.

In larger organizations, data tends to be stored in different places in different forms.
I am certainly a proponent of "single source of truth", but this is a difficult, possibly unrealistic, idea for many teams.
This is the situation my friend is in.
Some data is stored in [Netbox](https://github.com/netbox-community/netbox), some data is stored in another backend, and it is all glued together with an in-house API.
This isn't new territory for either of us; his question to me is, "There must be a better way!".
"Better way" is in the eye of the beer-holder, but let's consider a functional (as in functional programming) approach to the problem.

Here is the situation:
We need to build a device configuration from a template.
Network engineers and developers have been doing this for years now; it's a well-understood problem, right?
The issue is that managing exceptions can get a bit thorny.
The workflow is straightforward:

- Get a list of devices and metadata from Netbox (API call)
- Get more device data from our secondary source of truth
- Merge the data
- Render a template
- Output the template (write it to a file)

Cool, that looks as simple as a half-dozen function calls!
What about side effects?
What if the second API call fails?
What if a null field is returned from Netbox and we expect it to be a concrete value?
What if an I/O error occurs while loading the template from disk?

The [`returns`](https://github.com/dry-python/returns) library gives us the tools to elegantly handle these situations.
Let's take a look and see how we might go about performing this task...

Required libraries:

- [returns](https://github.com/dry-python/returns)
- [jinja2](https://jinja2docs.readthedocs.io/en/stable/index.html)
- [requests](https://docs.python-requests.org/en/master/index.html)

I will address each step of the workflow in the following sections and then tie them together at the end.
I will also be pointing out several places where things can break.
How often have you seen a 500 returned from Django or anther framework because the exception wasn't passed back far enough?
Hopefully, we can find a better way to handle these situations!

## Collect data from netbox

This is a simple API request that retrieves data from Netbox's demo site.

Here is our first dive into errors.
Try to think about all the ways this function can fail...

In [62]:
import requests
import typing as t

NETBOX_URL = "https://netboxdemo.com"
NETBOX_API_TOKEN = " 72830d67beff4ae178b94d8f781842408df8069d"

def get_netbox_devices() -> t.List[t.Any]:
    devices = requests.get(
        f"{NETBOX_URL}/api/dcim/devices/",
        headers={
            "Authorization": f"Token {NETBOX_API_TOKEN}",
            "Accept": "application/json"
        }
    ).json()["results"]
    return devices


### I/O failsauce

Here are some scenarios:

- DNS failure
- socket timeout
- Auth failure
- Empty response
- Key error (lookup "results")

Do we need to handle each one individually, or do we only care that something "unexpected" happened?
That peice of the puzzle is entirely up to you as a developer.
What I am interested in is how we chain a bunch of flaky tasks together.

## Get data from our secondary system

I don't have all the details on the actual system in use, but it doesn't matter.
I will simply emulate our requirements with an sqlite database.
This routine suffers from most of the same failure modes as the previous one, plus disk I/O issues.

The first part below is a mess of boilerplate that creates random data in our faux DB.
I am skipping building a real schema, this is a quick-and-dirty demo, after all.
You can find the useful bits at the bottom.

In [163]:
from dataclasses import dataclass
import socket
import sqlite3

@dataclass
class L3vniRow:
    vni: int
    addr_family: int
    ip_address: int
    netmask: int
    site: str

@dataclass
class DeviceIPAMRow:
    addr_family: int
    ip_address: int
    netmask: int
    interface: str
    device: str


con = sqlite3.connect(':memory:')
con.text_factory = bytes

def create_data(con) -> None:
    with con:
        con.execute('''CREATE TABLE l3vni
                    (vni INTEGER, addr_family INTEGER, ip_address BLOB, netmask INTEGER, site TEXT)''')
        con.executemany('''INSERT INTO l3vni VALUES (?, ?, ?, ?, ?)''',
            ((54321, 4, b"\n\x00d\x01", 24, "DS9"),
            (98765, 4, b"\nd\x00\x01", 24, "NCC-1701-D"))
        )
        con.execute('''CREATE TABLE device_ipam
                    (addr_family INTEGER, ip_address BLOB, netmask INTEGER, interface TEXT, device TEXT)''')
        con.executemany('''INSERT INTO device_ipam VALUES (?, ?, ?, ?, ?)''',
            ((4, b'\xc0\xa8\x00\xc2', 24, "loopback 5", "1701_CORE_SWITCH"),
            (4, b'\xac\x114\xcc', 24, "management 1", "1701_CORE_SWITCH"),
            (4, b'\xc0\xa8\x00\x0e', 24, "loopback 5", "1701_FW"),
            (4, b'\xac\x114.', 24, "management 1", "1701_FW"),
            (4, b'\xc0\xa8\x00\xf2', 24, "loopback 5", "1701_MDF_ACCESS-sw1"),
            (4, b'\xac\x114\x1b', 24, "management 1", "1701_MDF_ACCESS-sw1"),
            (4, b'\xc0\xa8\x00\x87', 24, "loopback 5", "1701_MDF_ACCESS-sw2"),
            (4, b'\xac\x114@', 24, "management 1", "1701_MDF_ACCESS-sw2"),
            (4, b'\xc0\xa8\x00\xdd', 24, "loopback 5", "1701_MDF_R1_PDU_A"),
            (4, b'\xac\x114\xf1', 24, "management 1", "1701_MDF_R1_PDU_A"),
            (4, b'\xc0\xa8\x00\x9d', 24, "loopback 5", "1701_MDF_R1_PDU_B"),
            (4, b'\xac\x114\xae', 24, "management 1", "1701_MDF_R1_PDU_B"),
            (4, b'\xc0\xa8\x00\xe9', 24, "loopback 5", "1701_MDF_R1_U40"),
            (4, b'\xac\x114\x11', 24, "management 1", "1701_MDF_R1_U40"),
            (4, b'\xc0\xa8\x000', 24, "loopback 5", "1701_MDF_R1_U41"),
            (4, b'\xac\x114\x8d', 24, "management 1", "1701_MDF_R1_U41"),
            (4, b'\xc0\xa8\x00\xfe', 24, "loopback 5", "1701_MDF_R1_U42"),
            (4, b'\xac\x114\xed', 24, "management 1", "1701_MDF_R1_U42"),
            (4, b'\xc0\xa8\x00\xb0', 24, "loopback 5", "1701_RR_ACCESS"),
            (4, b'\xac\x114\x9c', 24, "management 1", "1701_RR_ACCESS"),
            (4, b'\xc0\xa8\x00\xeb', 24, "loopback 5", "1701_RR_IDF_PDU"),
            (4, b'\xac\x114\xad', 24, "management 1", "1701_RR_IDF_PDU"),
            (4, b'\xc0\xa8\x006', 24, "loopback 5", "1701_RR_IDF_U7"),
            (4, b'\xac\x114\x85', 24, "management 1", "1701_RR_IDF_U7"),
            (4, b'\xc0\xa8\x00\xf6', 24, "loopback 5", "1701_RR_IDF_U8"),
            (4, b'\xac\x114\x19', 24, "management 1", "1701_RR_IDF_U8"),
            (4, b'\xc0\xa8\x00\xcf', 24, "loopback 5", "DS9_CORE_ACCESS"),
            (4, b'\xac\x114\xf3', 24, "management 1", "DS9_CORE_ACCESS"),
            (4, b'\xc0\xa8\x00a', 24, "loopback 5", "DS9_CORE_FIBER_U42"),
            (4, b'\xac\x114j', 24, "management 1", "DS9_CORE_FIBER_U42"),
            (4, b'\xc0\xa8\x004', 24, "loopback 5", "DS9_CORE_PDU_A"),
            (4, b'\xac\x114K', 24, "management 1", "DS9_CORE_PDU_A"),
            (4, b'\xc0\xa8\x00\xac', 24, "loopback 5", "DS9_CORE_PDU_B"),
            (4, b'\xac\x114\xf7', 24, "management 1", "DS9_CORE_PDU_B"),
            (4, b'\xc0\xa8\x00\xbf', 24, "loopback 5", "DS9_CORE_SWITCH"),
            (4, b'\xac\x114\x85', 24, "management 1", "DS9_CORE_SWITCH"),
            (4, b'\xc0\xa8\x00]', 24, "loopback 5", "DS9_CORE_U41"),
            (4, b'\xac\x114x', 24, "management 1", "DS9_CORE_U41"),
            (4, b'\xc0\xa8\x00\xe0', 24, "loopback 5", "DS9_FW"),
            (4, b'\xac\x114F', 24, "management 1", "DS9_FW"),
            (4, b'\xc0\xa8\x00\xaa', 24, "loopback 5", "DS9_QUARKS_ACCESS"),
            (4, b'\xac\x114\xd8', 24, "management 1", "DS9_QUARKS_ACCESS"),
            (4, b'\xc0\xa8\x00\x9a', 24, "loopback 5", "DS9_QUARKS_FIBER_U8"),
            (4, b'\xac\x114\x98', 24, "management 1", "DS9_QUARKS_FIBER_U8"),
            (4, b'\xc0\xa8\x00\xda', 24, "loopback 5", "DS9_QUARKS_PDU"),
            (4, b'\xac\x114^', 24, "management 1", "DS9_QUARKS_PDU"),
            (4, b'\xc0\xa8\x00}', 24, "loopback 5", "DS9_QUARKS_U7"),
            (4, b'\xac\x114_', 24, "management 1", "DS9_QUARKS_U7"),
            (4, b'\xc0\xa8\x00g', 24, "loopback 5", "DS9_SECOFFICE_ACCESS"),
            (4, b'\xac\x1145', 24, "management 1", "DS9_SECOFFICE_ACCESS"),
            (4, b'\xc0\xa8\x00\xca', 24, "loopback 5", "DS9_SECOFFICE_FIBER_U12"),
            (4, b'\xac\x1144', 24, "management 1", "DS9_SECOFFICE_FIBER_U12"),
            (4, b'\xc0\xa8\x00C', 24, "loopback 5", "DS9_SECOFFICE_PDU"),
            (4, b'\xac\x114\xfb', 24, "management 1", "DS9_SECOFFICE_PDU"),
            (4, b'\xc0\xa8\x00\xe6', 24, "loopback 5", "DS9_SECOFFICE_U11"),
            (4, b'\xac\x114\x10', 24, "management 1", "DS9_SECOFFICE_U11"),)
        )

# Create all the datas
create_data(con)

def get_af(af: int):
    if af == 4:
        family = socket.AF_INET
    elif af == 6:
        family = socket.AF_INET6
    return family

def l3vni_row_factory(cursor, row) -> L3vniRow:
    """Row factory to load l3vni data into an L3vniRow class."""
    return L3vniRow(row[0], get_af(row[1]), row[2], row[3], row[4].decode('utf-8'))

def get_vni_data(con) -> t.Dict[str, t.Any]:
    """Retreive vni data rows"""
    con.row_factory = l3vni_row_factory
    with con:
        data = con.execute("SELECT * FROM l3vni")
    site_data = {}
    for row in data:
        site_data[row.site] = row
    con.row_factory = None
    return site_data

def device_ipam_row_factory(cursor, row) -> DeviceIPAMRow:
    """Row factory to load l3vni data into an DeviceIPAMRow class."""
    return DeviceIPAMRow(get_af(row[0]), row[1], row[2], row[3].decode('utf-8'), row[4].decode('utf-8'))

def get_device_ipam_data(con) -> t.Dict[str, t.Any]:
    """Retreive device IPAM rows"""
    con.row_factory = device_ipam_row_factory
    with con:
        data = con.execute("SELECT * FROM device_ipam")
    device_data = {}
    for row in data:
        device_data[(row.device, row.interface)] = row
    con.row_factory = None
    return device_data


## Create a Jinja2 template

This is another familiar task in the infrastructure automation domain.
No magic here!
😁
I am not trying to create a working topology, this is simply a pet example (no judging! 😝).

In [164]:
from jinja2 import Environment, DictLoader, StrictUndefined

GLOBAL_VARS = {
    "NTP_SERVER_IP_ADDR": "1.1.1.1",
    "LOGIN_SECRET": "supersecret"
}

config_template = """hostname {{ hostname }}

ntp server {{ ntp_server_ip_addr }}

username nw-ops role network-admin secret 0 {{ login_secret }}

interface Loopback5
  ip address {{ lo5_cidr }}

interface vxlan 1
  vxlan source-interface loopback 5
  vxlan vlan 100 vni {{ site_vni }}

interface vlan 100
  description {{ site_svi_desc }}
  ip address {{ site_svi_cidr }}

interface Management1
  ip address {{ management_ip_cidr }}
  no shutdown
"""

env = Environment(
    loader=DictLoader({"config_template.j2": config_template}),
    undefined=StrictUndefined
)

template = env.get_template('config_template.j2')

# Sanity check that we're on the right path
vars = dict(
    hostname="s-sw-1",
    ntp_server_ip_addr="192.168.0.1",
    login_secret="foobar",
    lo5_cidr="1.1.1.1/32",
    site_svi_desc="Routed SVI for site TEST",
    site_vni=444444,
    site_svi_cidr="4.3.2.1/24",
    management_ip_cidr="172.17.0.42/28"
)
print(template.render(**vars))

hostname s-sw-1

ntp server 192.168.0.1

username nw-ops role network-admin secret 0 foobar

interface Loopback5
  ip address 1.1.1.1/32

interface vxlan 1
  vxlan source-interface loopback 5
  vxlan vlan 100 vni 444444

interface vlan 100
  description Routed SVI for site TEST
  ip address 4.3.2.1/24

interface Management1
  ip address 172.17.0.42/28
  no shutdown


## Level set

The previous bits of hacking are probably typical in a new workflow.
Find some demo data, figure out the relationships, create a template, see what happens.
TDD advocates are squirming in their chairs right now, but that's not the point of this post.

Let's tie this mess up into one big knot and see what happens!

In [167]:
from dataclasses import asdict

@dataclass
class TemplateVars:
    hostname: str
    lo5_cidr: str
    site_svi_desc: str
    site_vni: int
    site_svi_cidr: str
    management_ip_cidr: str
    ntp_server_ip_addr: str = GLOBAL_VARS["NTP_SERVER_IP_ADDR"]
    login_secret: str = GLOBAL_VARS["LOGIN_SECRET"]

def render_device_template() -> t.Dict[str, t.Any]:
    devices = get_netbox_devices()
    device_ipam = get_device_ipam_data(con)
    site_vni_data = get_vni_data(con)
    device_templates = {}
    for device in devices:
        hostname = device["name"]
        site_name = device["site"]["name"]
        lo5_meta = device_ipam[(hostname, "loopback 5")]
        mgmt_meta = device_ipam[(hostname, "management 1")]
        lo5_cidr = f'{socket.inet_ntop(lo5_meta.addr_family, lo5_meta.ip_address)}/{lo5_meta.netmask}'
        site_svi_desc = f"Routed SVI for site {site_name}"
        site_vni = site_vni_data[site_name].vni
        site_svi_cidr = f'{socket.inet_ntop(socket.AF_INET, site_vni_data[site_name].ip_address)}/{site_vni_data[site_name].netmask}'
        mgmt_cidr = f'{socket.inet_ntop(mgmt_meta.addr_family, mgmt_meta.ip_address)}/{mgmt_meta.netmask}'
        template_vars = TemplateVars(
            hostname=hostname,
            lo5_cidr=lo5_cidr,
            site_svi_desc=site_svi_desc,
            site_vni=site_vni,
            site_svi_cidr=site_svi_cidr,
            management_ip_cidr=management_ip_cidr
        )
        device_templates[hostname] = template.render(asdict(template_vars))
    return device_templates

templates = render_device_template()
print(templates['1701_MDF_ACCESS-sw1'])

hostname 1701_MDF_ACCESS-sw1

ntp server 1.1.1.1

username nw-ops role network-admin secret 0 supersecret

interface Loopback5
  ip address 192.168.0.242/24

interface vxlan 1
  vxlan source-interface loopback 5
  vxlan vlan 100 vni 98765

interface vlan 100
  description Routed SVI for site NCC-1701-D
  ip address 10.100.0.1/24

interface Management1
  ip address 172.17.52.16/24
  no shutdown
